## Synthetic Cholera and Typhoid dataset and Model for Cholera and Typhoid outbreak

This page has a synthetic(self-created/formulated) dataset to show how we can predict **Cholera** and **Typhoid** outbreaks using environmental and community factors.
The data is not real but helps us to demonstrate how an AI model would work if we had the actual specific data from different regions in Ghana.

### Features

**Region** : The region in Ghana where the data is from.

**City** : The area in Ghana where the data is from.

**Year** : The year the data is from.

**Month** : The month of the year.

**Rainfall(mm)**: Average rainfall in millimeters(mm) for that month.

**Temperature(Celsius)**: Average temperature in Celsius.

**Sanitation_Index**: A number between 0 and 100 showing how clean the area is. Lower number mean poor sanitation and vice versa.

**Water_Quality_Index**: A score between 0 and 100 that show how clean the area are.

**Population_Density**: Number of people living per square Kilometer.

**Waste_Management_Core**: A score between 0 and 100 showing how well waste is collected and disposed of.

**Cholera_Cases**: The number of cholera cases occured.

**Typhoid_Cases**: The number of typhoid cases occured.

**Next_Month_Cholera**: The number of predicted cholera cases for the next month.

**Next_Month_Typhoid**: The number of predicted typhoid cases for the next month.


### Training the Model
We use XGBoost, Random Forest, Gradient Boosting and Stacking Regressors within a multi-output pipeline to predict the number of Cholera and Typhoid cases for the next month.
This model learns patterns from the environmental and community features defined earlier.


## Generating the synthetic dataset

In [66]:
import pandas as pd
import numpy as np


In [68]:
# Seed
np.random.seed(42)

# Pairing cities with regions
city_region_map = { 'Accra': 'Greater Accra', 'Cape Coast': 'Central', 'Kumasi': 'Ashanti', 'Ho': 'Volta', 'Tamale': 'Northern', 'Takoradi': 'Western', 'Wa': 'Upper West', 'Bolgatanga': 'Upper East' }
# Number of data samples we want to generate
n_samples = 2000  

# Randomly pick cities from the city_region_map
cities = np.random.choice(list(city_region_map.keys()), size=n_samples)

# Get the region that matches each city
regions = [city_region_map[city] for city in cities]

# Randomly pick years and months
years = np.random.choice(range(2019, 2025), size=n_samples)
months = np.random.choice(range(1, 13), size=n_samples)

# Generate random values for environmental and community factors
rainfall = np.random.randint(50, 250, size=n_samples)
temperature = np.round(np.random.uniform(26, 33, size=n_samples), 1)
population_density = np.random.randint(100, 7000, size=n_samples)
sanitation = np.round(np.random.uniform(30, 90, size=n_samples), 2)
water_quality_index = np.round(np.random.uniform(30, 90, size=n_samples), 2)
waste_management_score = np.round(np.random.uniform(30, 90, size=n_samples), 2)

# Put everything into a DataFrame
df = pd.DataFrame({
    "Region": regions,
    "City": cities,
    "Year": years,
    "Month": months,
    "Rainfall_mm": rainfall,
    "Temperature_celsius": temperature,
    "Sanitation_Index": sanitation,
    "Water_Quality_Index": water_quality_index,
    "Population_Density": population_density,
    "Waste_Management_Score": waste_management_score
})

# Create synthetic cholera cases based on environmental factors
df["Cholera_Cases"] = ((df["Rainfall_mm"] / 12) + ((100 - df["Sanitation_Index"]) / 8) + (df["Population_Density"] / 900) + np.random.normal(0, 0.5, size=df.shape[0])).round().astype(int)

# Create synthetic typhoid cases based on other related factors
df["Typhoid_Cases"] = ((df["Rainfall_mm"] / 18) + ((100 - df["Water_Quality_Index"]) / 9) + (df["Population_Density"] / 1100) + np.random.normal(0, 0.5, size=df.shape[0])).round().astype(int)


In [69]:
# Shift the cases by one month within each city to get the next month's values.
# This helps us predict next month's cholera and typhoid cases based on current data.
df = df.sort_values(by=["City", "Year", "Month"])
df["Next_Month_Cholera"] = df.groupby("City")["Cholera_Cases"].shift(-1)
df["Next_Month_Typhoid"] = df.groupby("City")["Typhoid_Cases"].shift(-1)
df = df.dropna(subset=["Next_Month_Cholera", "Next_Month_Typhoid"])



In [ ]:
# First 5 rows in the dataframe
df.head()

,Region,City,Year,Month,Rainfall_mm,Temperature_celsius,Sanitation_Index,Water_Quality_Index,Population_Density,Waste_Management_Score,Cholera_Cases,Typhoid_Cases,Next_Month_Cholera,Next_Month_Typhoid
352,Greater Accra,Accra,2019,2,216,29.3,41.87,38.69,5205,35.15,31,24,30.0,17.0
148,Greater Accra,Accra,2019,3,226,29.9,42.25,77.58,3016,52.49,30,17,28.0,22.0
317,Greater Accra,Accra,2019,3,191,28.6,67.82,58.79,6911,86.14,28,22,28.0,19.0
1047,Greater Accra,Accra,2019,3,239,31.8,68.06,74.37,3916,48.02,28,19,33.0,23.0
1325,Greater Accra,Accra,2019,3,243,31.6,46.40,56.94,5095,63.61,33,23,32.0,21.0


In [71]:
# Gives a quick summary of the dataset like the mean, min, max, and quartiles for each column.
df.describe()

,Year,Month,Rainfall_mm,Temperature_celsius,Sanitation_Index,Water_Quality_Index,Population_Density,Waste_Management_Score,Cholera_Cases,Typhoid_Cases,Next_Month_Cholera,Next_Month_Typhoid
count,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000,1992.000000
mean,2021.526104,6.465361,152.244478,29.478665,60.022279,59.750095,3585.276606,60.170979,21.658635,16.196285,21.655120,16.188253
std,1.708642,3.458559,57.944170,2.026361,17.353431,17.304790,1985.053472,17.167216,5.727438,4.167286,5.717793,4.166693
min,2019.000000,1.000000,50.000000,26.000000,30.010000,30.060000,107.000000,30.010000,7.000000,5.000000,7.000000,5.000000
25%,2020.000000,3.000000,103.000000,27.700000,45.390000,45.230000,1801.250000,45.397500,17.000000,13.000000,17.000000,13.000000
50%,2022.000000,6.000000,154.000000,29.400000,60.285000,59.555000,3670.000000,60.550000,22.000000,16.000000,22.000000,16.000000
75%,2023.000000,9.000000,202.250000,31.300000,75.025000,74.492500,5283.750000,74.745000,26.000000,19.000000,26.000000,19.000000
max,2024.000000,12.000000,249.000000,33.000000,89.980000,89.980000,6990.000000,89.990000,35.000000,28.000000,35.000000,28.000000


In [72]:
# Information on the number of missing values, and datatype of each column
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1992 entries, 352 to 78
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Region                  1992 non-null   object 
 1   City                    1992 non-null   object 
 2   Year                    1992 non-null   int64  
 3   Month                   1992 non-null   int64  
 4   Rainfall_mm             1992 non-null   int32  
 5   Temperature_celsius     1992 non-null   float64
 6   Sanitation_Index        1992 non-null   float64
 7   Water_Quality_Index     1992 non-null   float64
 8   Population_Density      1992 non-null   int32  
 9   Waste_Management_Score  1992 non-null   float64
 10  Cholera_Cases           1992 non-null   int64  
 11  Typhoid_Cases           1992 non-null   int64  
 12  Next_Month_Cholera      1992 non-null   float64
 13  Next_Month_Typhoid      1992 non-null   float64
dtypes: float64(6), int32(2), int64(4), object(2)


## Data Preprocessing and Model Training

In [67]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [73]:
# Preprocessing data, filling with missing values and passing them through a pipeline and then through a column transformer.
numeric_features = [
    "Temperature_celsius", "Rainfall_mm", "Population_Density",
    "Water_Quality_Index", "Sanitation_Index", "Waste_Management_Score"
]
categorical_features = ["Region", "City"]

numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])


## Model Training

In [ ]:
# Split the data into input and target, i.e x and y, then splilt x for training and testing
X = df.drop(["Cholera_Cases", "Typhoid_Cases", "Next_Month_Cholera", "Next_Month_Typhoid"], axis=1)

y_multi = df[["Next_Month_Cholera", "Next_Month_Typhoid"]]

X_train, X_test, y_train, y_test = train_test_split(X, y_multi, test_size=0.2, random_state=42)


Extreme Boosting Regressor(XGB Regressor)

In [ ]:
# Build a pipeline that first preprocesses the data, and and then uses an XGBoost Regressor wrapped in a MultiOutput Regressor to predict future cholera and typhoid cases.
xgb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(XGBRegressor(random_state=42, n_estimators=300, learning_rate=0.1, max_depth=5)))
])

xgb_pipe.fit(X_train, y_train)
y_pred_xgb = xgb_pipe.predict(X_test)

In [ ]:
# Print RMSE and R2 score.
print("\n===== XGBoost Results =====")
print("Cholera → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_xgb[:, 0])))
print("Cholera → R²:", r2_score(y_test["Next_Month_Cholera"], y_pred_xgb[:, 0]))
print("Typhoid → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_xgb[:, 1])))
print("Typhoid → R²:", r2_score(y_test["Next_Month_Typhoid"], y_pred_xgb[:, 1]))


===== XGBoost Results =====
Cholera → RMSE: 6.10497595293681
Cholera → R²: -0.15127453486020248
Typhoid → RMSE: 4.448413314491459
Typhoid → R²: -0.14170687453519726


Random Forest Regressor

In [ ]:
# Build a pipeline that first preprocesses the data, and and then uses an Random Forest Regressor wrapped in an MultiOutput Regressor to predict future cholera and typhoid cases.
rf_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(random_state=42, n_estimators=200)))
])

rf_pipe.fit(X_train, y_train)
y_pred_rf = rf_pipe.predict(X_test)

In [ ]:
# Print RMSE and R2 score.
print("\n===== Random Forest Results =====")
print("Cholera → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_rf[:, 0])))
print("Cholera → R²:", r2_score(y_test["Next_Month_Cholera"], y_pred_rf[:, 0]))
print("Typhoid → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_rf[:, 1])))
print("Typhoid → R²:", r2_score(y_test["Next_Month_Typhoid"], y_pred_rf[:, 1]))


===== Random Forest Results =====
Cholera → RMSE: 5.7385121103472025
Cholera → R²: -0.017207581235789693
Typhoid → RMSE: 4.2701749581173285
Typhoid → R²: -0.052048328906511454


Gradient Boosting Regressor

In [ ]:
# Build a pipeline that first preprocesses the data, and and then uses an Gradient Boosting Regressor wrapped MultiOutput Regressor in a to predict future cholera and typhoid cases.
gb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(GradientBoostingRegressor(random_state=42)))
])

gb_pipe.fit(X_train, y_train)
y_pred_gb = gb_pipe.predict(X_test)

In [61]:
print("\n===== Gradient Boosting Results =====")
print("Cholera → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_gb[:, 0])))
print("Cholera → R²:", r2_score(y_test["Next_Month_Cholera"], y_pred_gb[:, 0]))
print("Typhoid → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_gb[:, 1])))
print("Typhoid → R²:", r2_score(y_test["Next_Month_Typhoid"], y_pred_gb[:, 1]))


===== Gradient Boosting Results =====
Cholera → RMSE: 5.742481135163961
Cholera → R²: -0.0186151650334625
Typhoid → RMSE: 4.1951663439734
Typhoid → R²: -0.015413010860267917


Stacking Regressor

In [ ]:
# Build a pipeline that first preprocesses the data, and and then uses an Stacking Regressor wrapped in an MultiOutput Regressor to predict future cholera and typhoid cases.
stacking_regressor = StackingRegressor(
    estimators=[
        ('xgb', XGBRegressor(random_state=42, n_estimators=300, learning_rate=0.1, max_depth=5)),
        ('rf', RandomForestRegressor(random_state=42, n_estimators=200)),
        ('gb', GradientBoostingRegressor(random_state=42)),
        ('hgb', HistGradientBoostingRegressor(random_state=42))
    ],
    final_estimator=XGBRegressor(random_state=42, learning_rate=0.1, n_estimators=100)
)

blend_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(stacking_regressor))
])

blend_pipe.fit(X_train, y_train)
y_pred_blend = blend_pipe.predict(X_test)

In [63]:
print("\n===== Blended Ensemble (Stacking) =====")
print("Cholera → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_blend[:, 0])))
print("Cholera → R²:", r2_score(y_test["Next_Month_Cholera"], y_pred_blend[:, 0]))
print("Typhoid → RMSE:", np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_blend[:, 1])))
print("Typhoid → R²:", r2_score(y_test["Next_Month_Typhoid"], y_pred_blend[:, 1]))


===== Blended Ensemble (Stacking) =====
Cholera → RMSE: 5.917636736980268
Cholera → R²: -0.08170188609550522
Typhoid → RMSE: 4.430398956098188
Typhoid → R²: -0.13247865273063986


### Choosing the best model

In [ ]:
# Store results from each model
results = pd.DataFrame({
    "Model": ["XGBoost", "Random Forest", "Gradient Boosting", "Blended Ensemble"],
    "Cholera_RMSE": [
        np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_xgb[:, 0])),
        np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_rf[:, 0])),
        np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_gb[:, 0])),
        np.sqrt(mean_squared_error(y_test["Next_Month_Cholera"], y_pred_blend[:, 0]))
    ],
    "Cholera_R2": [
        r2_score(y_test["Next_Month_Cholera"], y_pred_xgb[:, 0]),
        r2_score(y_test["Next_Month_Cholera"], y_pred_rf[:, 0]),
        r2_score(y_test["Next_Month_Cholera"], y_pred_gb[:, 0]),
        r2_score(y_test["Next_Month_Cholera"], y_pred_blend[:, 0])
    ],
    "Typhoid_RMSE": [
        np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_xgb[:, 1])),
        np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_rf[:, 1])),
        np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_gb[:, 1])),
        np.sqrt(mean_squared_error(y_test["Next_Month_Typhoid"], y_pred_blend[:, 1]))
    ],
    "Typhoid_R2": [
        r2_score(y_test["Next_Month_Typhoid"], y_pred_xgb[:, 1]),
        r2_score(y_test["Next_Month_Typhoid"], y_pred_rf[:, 1]),
        r2_score(y_test["Next_Month_Typhoid"], y_pred_gb[:, 1]),
        r2_score(y_test["Next_Month_Typhoid"], y_pred_blend[:, 1])
    ]
})

# Display comparison table
print("===== Model Performance Comparison =====")
print(results)

# Choose the best model based on lowest average RMSE
results["Avg_RMSE"] = (results["Cholera_RMSE"] + results["Typhoid_RMSE"]) / 2
best_model_row = results.loc[results["Avg_RMSE"].idxmin()]

print("\n🏆 Best Overall Model Based on RMSE:")
print(best_model_row[["Model", "Cholera_RMSE", "Typhoid_RMSE", "Avg_RMSE"]])


===== Model Performance Comparison =====
               Model  Cholera_RMSE  Cholera_R2  Typhoid_RMSE  Typhoid_R2
0            XGBoost      6.104976   -0.151275      4.448413   -0.141707
1      Random Forest      5.738512   -0.017208      4.270175   -0.052048
2  Gradient Boosting      5.742481   -0.018615      4.195166   -0.015413
3   Blended Ensemble      5.917637   -0.081702      4.430399   -0.132479

🏆 Best Overall Model Based on RMSE:
Model           Gradient Boosting
Cholera_RMSE             5.742481
Typhoid_RMSE             4.195166
Avg_RMSE                 4.968824
Name: 2, dtype: object


Therefore the choosen model is the Gradient Boosting Regressor model.

### Exporting the model

In [65]:
# pipe = your trained pipeline
joblib.dump(gb_pipe, 'cholera_gb_pipeline.joblib')
print("Model saved successfully!")


Model saved successfully!
